<a href="https://colab.research.google.com/github/danielscp/semantic_lexical_lir/blob/master/Models%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/danielscp/semantic_lexical_lir.git

Cloning into 'semantic_lexical_lir'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 225 (delta 84), reused 210 (delta 71), pack-reused 0
Receiving objects: 100% (225/225), 462.30 MiB | 23.00 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Checking out files: 100% (158/158), done.


In [0]:
!pip install gensim

In [0]:
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import TfidfModel, LsiModel, LdaModel

In [0]:
corpus_a = {
    'models_dir': 'semantic_lexical_lir/data/corpus_a/models/',
    'data_dir': 'semantic_lexical_lir/data/corpus_a/',
    'output_dir': 'corpus_a/'
}

corpus_b = {
    'models_dir': 'semantic_lexical_lir/data/corpus_b/models/',
    'data_dir': 'semantic_lexical_lir/data/corpus_b/',
    'output_dir': 'corpus_b/'
}

In [0]:
load_bows = lambda c: MmCorpus(c['data_dir'] + 'bows')
corpus_a['bows'] = load_bows(corpus_a)
corpus_b['bows'] = load_bows(corpus_b)

load_test_bows = lambda c: MmCorpus(c['data_dir'] + 'bows_test')
corpus_a['test_bows'] = load_test_bows(corpus_a)
corpus_b['test_bows'] = load_test_bows(corpus_b)


load_dict = lambda c: Dictionary.load(c['data_dir'] + 'dictionary.dict')
corpus_a['dict'] = load_dict(corpus_a)
corpus_b['dict'] = load_dict(corpus_b)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
tfidf_fit = lambda c: TfidfModel(c['bows'])
tfidf_transform = lambda c: c['tfidf']['model'][c['bows']]

corpus_a['tfidf'] = {'model': tfidf_fit(corpus_a)}
corpus_a['tfidf']['vects'] = tfidf_transform(corpus_a)

corpus_b['tfidf'] = {'model': tfidf_fit(corpus_b)}
corpus_b['tfidf']['vects'] = tfidf_transform(corpus_b)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 4.81 s, sys: 47.3 ms, total: 4.86 s
Wall time: 4.82 s


In [0]:
topics = [50, 100, 150, 200, 250, 300, 350, 400]

In [0]:
%%time
lsi_fit = lambda c, t: LsiModel(corpus=c['tfidf']['vects'], num_topics=t, id2word=c['dict'])
corpus_a['lsi'] = {t: lsi_fit(corpus_a, t) for t in topics}
corpus_b['lsi'] = {t: lsi_fit(corpus_b, t) for t in topics}

#lsi = {t: LsiModel(corpus=tfidf_vects, num_topics=t, id2word=dictionary) for t in topics}

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 3min 19s, sys: 10.1 s, total: 3min 29s
Wall time: 3min 1s


In [0]:
%%time
#Takes 1h 10min to train using Google Colaboratory GPU...
lda_fit = lambda c, t: LdaModel(corpus=c['bows'], num_topics=t, id2word=c['dict'], passes=15, eval_every=None)
# lda = {t: LdaModel(corpus=bows, num_topics=t, id2word=dictionary, passes=15, eval_every=None) for t in topics}
corpus_a['lda'] = {t: lda_fit(corpus_a, t) for t in topics}
corpus_b['lda'] = {t: lda_fit(corpus_b, t) for t in topics}

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 1h 35min 44s, sys: 37min 28s, total: 2h 13min 12s
Wall time: 1h 9min 41s


In [0]:
from semantic_lexical_lir.bm25 import BM25

In [0]:
corpus_a['bm25'] = BM25(corpus_a['test_bows'])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
